# 二维卷积层

## 二维互相关运算

In [2]:
import torch
import torch.nn as nn

In [3]:
def corr2d(X, K):
    h, w = K.shape
    Y = torch.zeros(X.shape[0] - h + 1, X.shape[1] - w + 1, device=X.device, dtype=X.dtype)
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

## 二维卷积层

In [4]:
class Conv2d(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.rand(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

## 图像的简单边缘检测

In [5]:
X = torch.ones(6, 8)
X[:, 2:6] = 0
X


tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [6]:
K = torch.tensor([[1, -1]])
Y = corr2d(X, K)
Y


tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

## 通过数据学习核数组

In [8]:
# 构造一个核数组形状是(1, 2)的二维卷积层
conv2d = Conv2d(kernel_size=(1, 2))

step = 20
lr = 0.01
for i in range(step):
    Y_hat = conv2d(X)
    l = ((Y_hat - Y) ** 2).sum()
    l.backward()

    # 梯度下降
    conv2d.weight.data -= lr * conv2d.weight.grad
    conv2d.bias.data -= lr * conv2d.bias.grad

    # 梯度清0
    conv2d.weight.grad.fill_(0)
    conv2d.bias.grad.fill_(0)
    if (i + 1) % 5 == 0:
        print('Step %d, loss %.3f' % (i + 1, l.item()))
        print("weight: ", conv2d.weight.data)
        print("bias: ", conv2d.bias.data)
        print('---')



Step 5, loss 3.030
weight:  tensor([[ 0.7486, -0.4941]])
bias:  tensor([-0.1433])
---
Step 10, loss 0.709
weight:  tensor([[ 0.8428, -0.7576]])
bias:  tensor([-0.0477])
---
Step 15, loss 0.182
weight:  tensor([[ 0.9088, -0.8803]])
bias:  tensor([-0.0159])
---
Step 20, loss 0.049
weight:  tensor([[ 0.9491, -0.9396]])
bias:  tensor([-0.0053])
---
